In [18]:
import numpy as np
from ultralytics import YOLO
from detections import Tracker, Detector, KeypointDetector
from calculations import TeamAffiliation, PlayerPosition
from utils import utils
from visualisation import Radar, VideoGenerator

In [2]:
SOURCE_PATH = r"C:\Users\Marcin\Videos\ganzgut.mp4"
DEST_PATH = r"C:\Users\Marcin\Videos\ganzgut_rgb_f.mp4"

In [19]:
model = YOLO("../training/runs/detect/train4/weights/best.pt")
model_keypoints = YOLO("../training/runs/pose/train8/weights/best.pt")
detector = Detector(model, 0.5)
keypoint_detector = KeypointDetector(model_keypoints)
tracker = Tracker(detector)
positions = PlayerPosition()

In [20]:
images_path = r'../data/SoccerNetGS/gamestate-2024/test/SNGS-118/img1'
frames = utils.load_frames_from_images(images_path)
#frames = utils.load_frames_from_video(SOURCE_PATH)

In [21]:
tracks = tracker.track(frames)


0: 384x640 10 players, 1 ball, 37.0ms
Speed: 24.0ms preprocess, 37.0ms inference, 26.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 players, 38.0ms
Speed: 4.0ms preprocess, 38.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 players, 1 ball, 35.0ms
Speed: 1.0ms preprocess, 35.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 players, 1 ball, 33.0ms
Speed: 2.0ms preprocess, 33.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 players, 32.0ms
Speed: 2.0ms preprocess, 32.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 players, 1 ball, 31.0ms
Speed: 2.0ms preprocess, 31.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 players, 1 ball, 19.0ms
Speed: 3.0ms preprocess, 19.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 players, 11.0ms
Speed: 2.0ms preprocess, 11.0ms

In [22]:
keypoints = keypoint_detector.detect(frames)


0: 384x640 1 pitch, 60.0ms
Speed: 4.0ms preprocess, 60.0ms inference, 12.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 32.0ms
Speed: 2.0ms preprocess, 32.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 34.0ms
Speed: 3.0ms preprocess, 34.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 37.0ms
Speed: 2.0ms preprocess, 37.0ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 33.0ms
Speed: 2.0ms preprocess, 33.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 11.0ms
Speed: 1.0ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

In [23]:
keypoints_list = keypoint_detector.fill_blank(frames)

In [24]:
player_positions = positions.get_players_position(keypoints_list, tracks)

In [25]:
teams = TeamAffiliation(tracks, frames)

In [26]:
shirt_colors, rgb_colors = teams.get_shirt_colors()

In [27]:
player_teams: list[np.array] = teams.get_rgb_teams()

In [28]:
team_colors = teams.get_average_team_color(player_teams,rgb_colors)

In [29]:
position_df = utils.parse_player_data(player_positions, tracks, player_teams)

In [31]:
from  utils import utils
utils.save_tracks_as_json(tracks, 'tracks-118.json')
position_df.to_csv('positions-118.csv')
utils.keypoints_to_json(keypoints, 'keypoints-118.json')

In [14]:
video = teams.annotate_frames()

In [15]:
radar = Radar(position_df, teams.get_team_colors())

In [16]:
video_generator = VideoGenerator(video, radar)

In [17]:
video_generator.generate_video_with_radar(DEST_PATH, n_frames_back=45)

100%|██████████| 553/553 [01:00<00:00,  9.15it/s]

Video saved to C:\Users\Marcin\Videos\ganzgut_rgb_f.mp4
